# [626. Exchange Seats](https://leetcode.com/problems/exchange-seats/description/?envType=study-plan-v2&envId=top-sql-50)

Table: Seat

<pre>+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| id          | int     |
| student     | varchar |
+-------------+---------+</pre>
id is the primary key (unique value) column for this table.
Each row of this table indicates the name and the ID of a student.
id is a continuous increment.
 

Write a solution to swap the seat id of every two consecutive students. If the number of students is odd, the id of the last student is not swapped.

Return the result table ordered by id in ascending order.

The result format is in the following example.

 

Example 1:

Input: 
Seat table:
<pre>+----+---------+
| id | student |
+----+---------+
| 1  | Abbot   |
| 2  | Doris   |
| 3  | Emerson |
| 4  | Green   |
| 5  | Jeames  |
+----+---------+</pre>
Output: 
<pre>+----+---------+
| id | student |
+----+---------+
| 1  | Doris   |
| 2  | Abbot   |
| 3  | Green   |
| 4  | Emerson |
| 5  | Jeames  |
+----+---------+</pre>
Explanation: 
Note that if the number of students is odd, there is no need to change the last one's seat.

In [1]:
#pandas schema

import pandas as pd

data = [[1, 'Abbot'], [2, 'Doris'], [3, 'Emerson'], [4, 'Green'], [5, 'Jeames']]
seat = pd.DataFrame(data, columns=['id', 'student']).astype({'id': 'Int64', 'student': 'object'})

In [2]:
#to spark dataframe

from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = SparkSession.builder.getOrCreate()

seat_df = spark.createDataFrame(seat)
seat_df.show()

bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
23/11/05 15:32:02 WARN Utils: Your hostname, svchost resolves to a loopback address: 127.0.1.1; using 172.18.28.34 instead (on interface eth0)
23/11/05 15:32:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/05 15:32:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+---+-------+
| id|student|
+---+-------+
|  1|  Abbot|
|  2|  Doris|
|  3|Emerson|
|  4|  Green|
|  5| Jeames|
+---+-------+


In [3]:
max_id = seat_df.agg(F.max('id')).collect()[0][0]

seat_df.withColumn('id',
                   F.when(F.col('id') % 2 == 0, F.col('id') - 1)
                   .when(F.col('id') == max_id, F.col('id'))
                   .otherwise(F.col('id') + 1)) \
    .orderBy('id') \
    .show()

+---+-------+
| id|student|
+---+-------+
|  1|  Doris|
|  2|  Abbot|
|  3|  Green|
|  4|Emerson|
|  5| Jeames|
+---+-------+


In [4]:
# In spark SQL

seat_df.createOrReplaceTempView('seat')

spark.sql('''
with max_id as (
    select max(id) as max_id from seat
)
select 
    case 
        when id%2=0 then id-1
        when id=(select max_id from max_id) then id
        else id+1 end as id,
    student
from seat
order by id;
''').show()

+---+-------+
| id|student|
+---+-------+
|  1|  Doris|
|  2|  Abbot|
|  3|  Green|
|  4|Emerson|
|  5| Jeames|
+---+-------+


In [ ]:
# using subquery in spark sql

spark.sql('''
select 
    case 
        when id%2=0 then id-1
        when id=(select max(id) from seat) then id
        else id+1 end as id,
    student
from seat
order by id;
''').show()

In [ ]:
spark.stop()